# Local-Global-Trend(LGT) Example

In [1]:
import pandas as pd
import numpy as np
from orbit.lgt import LGT
from orbit.utils.utils import plot_predicted_data

## Data

In [2]:
DATA_FILE = "./data/iclaims.example.csv"

In [3]:
raw_df = pd.read_csv(DATA_FILE, parse_dates=['week'])

In [4]:
raw_df.dtypes

week              datetime64[ns]
claims                     int64
trend.unemploy           float64
trend.filling            float64
trend.job                float64
dtype: object

In [5]:
raw_df.head(5)

,week,claims,trend.unemploy,trend.filling,trend.job
0,2010-01-03,651215,1.183973,0.720140,1.119669
1,2010-01-10,825891,1.183973,0.814896,1.178599
2,2010-01-17,659173,1.203382,0.739091,1.119669
3,2010-01-24,507651,1.164564,0.814896,1.107883
4,2010-01-31,538617,1.086926,0.776993,1.072525


### Train / Test Split

In [6]:
df=raw_df.copy()
test_size=52
train_df=df[:-test_size]

# in uTS we make assertion that test dates need to be between train start/end
test_df=df[-test_size:]

In [7]:
train_df.head(5)

,week,claims,trend.unemploy,trend.filling,trend.job
0,2010-01-03,651215,1.183973,0.720140,1.119669
1,2010-01-10,825891,1.183973,0.814896,1.178599
2,2010-01-17,659173,1.203382,0.739091,1.119669
3,2010-01-24,507651,1.164564,0.814896,1.107883
4,2010-01-31,538617,1.086926,0.776993,1.072525


## MAP Fit and Predict

In [8]:
lgt_map=LGT(
    response_col="claims",
    date_col="week",
    seasonality=52,
    seed=8888,
    predict_method='map',
    auto_scale=False,
    is_multiplicative=True
)

In [9]:
# for x in lgt_map._stan_input_mapper:
#     print(x)
#     print(x.value)
#     print(lgt_map._stan_input_mapper[x])

# gg = lgt_map._stan_input_mapper
# dict(lgt_map._stan_input_mapper)
for key in lgt_map._stan_input_mapper.__members__.keys():
    print(key)


NUM_OF_OBSERVATIONS
RESPONSE
SEASONALITY
SEASONALITY_MIN
SEASONALITY_MAX
SEASONALITY_SMOOTHING_MIN
SEASONALITY_SMOOTHING_MAX
GLOBAL_TREND_COEF_MIN
GLOBAL_TREND_COEF_MAX
GLOBAL_TREND_POW_MIN
GLOBAL_TREND_POW_MAX
LOCAL_TREND_COEF_MIN
LOCAL_TREND_COEF_MAX
LEVEL_SMOOTHING_MIN
LEVEL_SMOOTHING_MAX
SLOPE_SMOOTHING_MIN
SLOPE_SMOOTHING_MAX
MIN_NU
MAX_NU
CAUCHY_SD
FIX_REGRESSION_COEF_SD
REGRESSOR_SIGMA_SD
REGRESSION_COEF_MAX
NUM_OF_POSITIVE_REGRESSORS
POSITIVE_REGRESSOR_MATRIX
POSITIVE_REGRESSOR_BETA_PRIOR
POSITIVE_REGRESSOR_SIGMA_PRIOR
NUM_OF_REGULAR_REGRESSORS
REGULAR_REGRESSOR_MATRIX
REGULAR_REGRESSOR_BETA_PRIOR
REGULAR_REGRESSOR_SIGMA_PRIOR


In [10]:
lgt_map.fit(df=train_df)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
predicted_df = lgt_map.predict(df=test_df)

In [11]:
plot_predicted_data(training_actual_df=train_df, predicted_df=predicted_df, 
                    date_col=lgt_map.date_col, actual_col=lgt_map.response_col, pred_col='prediction', 
                    test_actual_df=test_df)

NameError: name 'predicted_df' is not defined

In [12]:
test_df.head(5)

,week,claims,trend.unemploy,trend.filling,trend.job
391,2017-07-02,252886,1.183973,1.099162,0.919307
392,2017-07-09,284329,1.222792,1.023357,1.025381
393,2017-07-16,257763,1.145154,1.061259,0.990023
394,2017-07-23,220455,1.125745,1.023357,1.025381
395,2017-07-30,198776,1.145154,1.042308,1.048953


## MCMC Fit and Predict

In [ ]:
lgt_mcmc = LGT(
    response_col='claims',
    date_col='week',
    seasonality=52,
    num_warmup=4000, 
    num_sample=500,
    sample_method='mcmc',
    predict_method='full',
    n_bootstrap_draws=500
)

In [ ]:
lgt_mcmc.fit(df=train_df)

In [ ]:
predicted_df = lgt_mcmc.predict(df=test_df)

In [ ]:
predicted_df.tail(5)

In [ ]:
plot_predicted_data(training_actual_df=train_df, predicted_df=predicted_df, 
                    date_col=lgt_mcmc.date_col, actual_col=lgt_mcmc.response_col, pred_col=50, 
                    pred_quantiles_col=[5,95], test_actual_df=test_df)

## Mean Fit and Predict Decomp

In [ ]:
lgt_mean=LGT(
    response_col='claims',
    date_col='week',
    seasonality=52,
    seed=8888,
    predict_method='mean'
)

In [ ]:
lgt_mean.fit(df=train_df)

In [ ]:
predicted_df = lgt_mean.predict(df=test_df, decompose=True)

In [ ]:
predicted_df.head(5)

In [ ]:
plot_predicted_data(training_actual_df=train_df, predicted_df=predicted_df, 
                    date_col=lgt_mean.date_col, actual_col=lgt_mean.response_col, pred_col='prediction', 
                    test_actual_df=test_df)

## MAP Fit and Decomp with Regression

In [ ]:
lgt_reg=LGT(
    response_col='claims',
    regressor_col=['trend.unemploy', 'trend.filling'],
    date_col='week',
    seasonality=52,
    seed=8888,
    predict_method='map'
)

In [ ]:
lgt_reg.fit(df=train_df)

In [ ]:
predicted_df = lgt_reg.predict(df=test_df, decompose=True)

In [ ]:
predicted_df.head(5)